In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import copy

import numpy as np 

from astropy.io import fits
from astropy.table import Table, Column

import matplotlib.pyplot as plt

## Learning about the `CP` reduced LAGER survey data

- Detailed description of community pipeline can be found [here](https://www.noao.edu/noao/staff/fvaldes/CPDocPrelim/PL201_3.html)

- There are currently four classes of data products: calibrations, instrumentally calibrated single exposures (non-remapped and remapped), and stacked (two version of background subtraction). Each of these consist of the basic image flux data plus various ancillary files associated with the flux data. The current ancillary file types are data quality, weight, and coverage/exposure maps.
- All data products are packaged as compressed, multi-extensions FITS (MEF) files. The extensions are either for each CCD or for tiles of a stacked data product. **DEAD CCDs are excluded; Overscan areas are trimmed**.

- `PRODTYPE`: The product type has five values 
    - `image/image1` for flux data.
    - `dqmask` for data quality maps
    - `wtmap` for weight map
    - `expmap` for exposure or coverage map. 
    - The principle type of scientific interest is the `image` type while the others provide ancillary data about the flux pixels.
    
- `PROCTYPE`: The processing type has four values
    - `MasterCal` for master calibrations of various observation types
    - `InstCal` for the instrumentally calibrated but not resampled single exposures
    - `Resampled` for the resampled single exposures
    - `Stacked` for stacked or coadded data from multiple exposures of the same pointing. 

- `OBSTYPE`: The observation type has five values 
    - `zero` for zero second or bias master calibrations
    - `dome flat` for master dome flat calibrations
    - `fringecor` for master fringe templates
    - `illumcor` for master illumination corrections
    - `object` for on-sky science data.
    
### Unremapped Science Exposures (`InstCal`)

- The unremapped science exposures are the ones to use if one wants to **keep the original CCD sampling**. 
- If the keyword `WCSCAL` is `Successful` then the images have a calibrated **TPV WCS** and the `RA` and `DEC` keywords are an accurate measure of the field center (as defined by the `CRPIX` keywords). 
- As noted in the calibration section, the flux values are calibrated assuming a constant pixel area projection on the sky.

### Remapped Science Exposures (`Resampled`)

- The resampled exposures are formated as MEF files where each CCD has been resampled to a common projection, tangent point, orientation, and pixel scale. 
- The projection is **a tangent plane** with tangent point given by the `CRVAL/CRPIX` keywords as per FITS WCS definitions. 
- The orientation is with RA decreasing as the column increases and DEC increasing as the line increases. 
- The pixel scale is **0.27 arcseconds per pixel**. 
- The number of pixels per axis is such that all data from the CCD is present and, because of the resampling, **there are non-physical pixels at the edges**.
- The `RA` and `DEC` keywords are the same as in the unresampled exposures so it represents the center of the field of view and not the tangent point.
- There are three files per exposure. These are the photon data, a data quality maps, and a weight map. The data quality and weight maps indicate the location of the non-physical edge pixels as well as pixels which have been flagged as non-scientific. - Currently **the data quality and weight maps for resampled exposures** where multiple exposures were taken is the only place where cosmic rays and other "transient" features found by the CP are identified.


In [2]:
data_dir = '/Users/song/Downloads/merian/'

lager_dir = os.path.join(data_dir, 'lager')

img_file = os.path.join(lager_dir, 'c4d_151208_074244_ooi_N964_v2.fits.fz')
dqa_file = os.path.join(lager_dir, 'c4d_151208_074244_ood_N964_v2.fits.fz')
wgt_file = os.path.join(lager_dir, 'c4d_151208_074244_oow_N964_v2.fits.fz')

In [7]:
fits.open(img_file)[0].header

SIMPLE  =                    T / Fits standard                                  
BITPIX  =                   16 / Bits per pixel                                 
NAXIS   =                    0 / Number of axes                                 
EXTEND  =                    T / File may contain extensions                    
DATE    = '2018-02-10T00:29:15' / Date FITS file was generated                  
IRAF-TLM= '2018-02-10T00:34:02' / Time of last modification                     
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
COMMENT MODIFIED:DATE,DATE-OBS,DTACQNAM,DTCALDAT,DTCOPYRI,DTINSTRU,DTNSANAM,DTOB
COMMENT SERV,DTPI,DTPIAFFL,DTPROPID,DTSITE,DTTELESC,DTTITLE,INSTRUME,OBSERVAT,OB
COMMENT SID,OBSTYPE,ODATEOBS,PROCTYPE,PRODTYPE,PROPID,SIMPLE,TELESCOP,TIME-OBS  
OBJECT  = 'COSMOS  '        